In [1]:
# Import the module
import pandas as pd
import numpy as np
import os

In [2]:
os.chdir('/home/kamran/CZ_Credit_Score/CZ_Data/CZ_FINAL_DATA')

# Main table = trans_six_months

In [3]:
# Loand the Data trans_six_months.csv
df = pd.read_csv('trans_six_months.csv')

In [4]:
df.head()

,account_id,date_trans,type,operation,amount,date_loan,status,days
0,10001,1996-01-14,credit,withdrawal_in_cash,500.0,1996-07-11,0.0,179
1,10001,1996-01-17,credit,withdrawal_in_cash,5700.0,1996-07-11,0.0,176
2,10001,1996-01-23,credit,withdrawal_in_cash,8500.0,1996-07-11,0.0,170
3,10001,1996-01-26,credit,withdrawal_in_cash,2500.0,1996-07-11,0.0,167
4,10001,1996-01-29,credit,withdrawal_in_cash,1900.0,1996-07-11,0.0,164


In [12]:
#Convert the data feature into proper data time format
df.date_trans = df.date_trans.apply(lambda x : pd.to_datetime(x))

In [6]:
df.date_trans.dt.year.value_counts()

1997    8192
1996    5127
1998    4685
1994    4441
1995    3329
1993    1228
Name: date_trans, dtype: int64

# 1. Find the count of Credit and withdrawl for each customer - Table name =type_count

In [7]:
# Find the count of Credit and withdrawl for each customer - Table name =type_count_raw
type_count_raw = df.groupby(['account_id', 'type'], squeeze = True).count().reset_index()[['account_id','type','date_trans']]

In [8]:
type_count_raw.head()

,account_id,type,date_trans
0,2,VYBER,3
1,2,credit,26
2,2,withdrawal,12
3,19,credit,20
4,19,withdrawal,22


In [9]:
type_count = type_count_raw.pivot(index='account_id', columns='type', values='date_trans').reset_index()

In [11]:
type_count.head()

type,account_id,VYBER,credit,withdrawal
0,2,3.0,26.0,12.0
1,19,NaN,20.0,22.0
2,25,1.0,37.0,16.0
3,37,1.0,39.0,12.0
4,38,1.0,30.0,16.0


In [19]:
type_count.columns

Index([u'account_id', u'VYBER', u'credit', u'withdrawal'], dtype='object', name=u'type')

In [21]:
type_count.dtypes

type
account_id      int64
VYBER         float64
credit        float64
withdrawal    float64
dtype: object

In [23]:
# Repalce NAN with 0
type_count.fillna(value = 0, inplace = True)

In [24]:
# Create the new column call total_type_count
type_count['total_type_count'] = type_count['VYBER'] +  type_count['credit'] + type_count['withdrawal']

In [26]:
type_count.head()

type,account_id,VYBER,credit,withdrawal,total_type_count
0,2,3.0,26.0,12.0,41.0
1,19,0.0,20.0,22.0,42.0
2,25,1.0,37.0,16.0,54.0
3,37,1.0,39.0,12.0,52.0
4,38,1.0,30.0,16.0,47.0


# 2. Find the average of each of the operation for each customer - Table name = oper_avg

In [62]:
oper_avg = (pd.pivot_table(df, index = 'account_id', columns= 'operation', values = 'amount',
                        aggfunc = np.average)).reset_index()


In [63]:
oper_avg.head()

operation,account_id,collection_from_another_bank,credit_card_withdrawal,credit_in_cash,other,remittance_to_another_bank,withdrawal_in_cash
0,2,21922.333333,NaN,NaN,143.616667,7266.000000,4729.775000
1,19,16400.500000,NaN,2966.666667,77.040000,NaN,6068.650000
2,25,NaN,NaN,25006.500000,264.283333,2597.812500,11062.072727
3,37,NaN,NaN,25244.666667,117.650000,1666.000000,6104.181818
4,38,14994.500000,NaN,2025.000000,156.300000,2282.384615,4367.555556


# 3. Merge the Table type_count and oper_avg name as type_oper_avg

In [64]:
type_oper_avg = pd.merge(type_count, oper_avg, how='right', on='account_id')

In [65]:
type_oper_avg.head()

,account_id,VYBER,credit,withdrawal,total_type_count,collection_from_another_bank,credit_card_withdrawal,credit_in_cash,other,remittance_to_another_bank,withdrawal_in_cash
0,2,3.0,26.0,12.0,41.0,21922.333333,NaN,NaN,143.616667,7266.000000,4729.775000
1,19,0.0,20.0,22.0,42.0,16400.500000,NaN,2966.666667,77.040000,NaN,6068.650000
2,25,1.0,37.0,16.0,54.0,NaN,NaN,25006.500000,264.283333,2597.812500,11062.072727
3,37,1.0,39.0,12.0,52.0,NaN,NaN,25244.666667,117.650000,1666.000000,6104.181818
4,38,1.0,30.0,16.0,47.0,14994.500000,NaN,2025.000000,156.300000,2282.384615,4367.555556


# 4. From main table drop the duplicate row on the basis of account_id and keep last or first, and have column account_id and status. table name loan_acc_status

In [66]:
loan_acc_status = df.drop_duplicates(subset='account_id', keep='first')[['account_id', 'status']]

In [67]:
loan_acc_status.head()

,account_id,status
0,10001,0.0
36,10005,0.0
69,10019,0.0
95,10022,0.0
128,1004,0.0


# 5. Merge type_oper_avg and loan_acc_status

In [68]:
trans_loan_final_six_months = pd.merge(type_oper_avg, loan_acc_status, how='left', on='account_id',sort=True)

In [69]:
trans_loan_final_six_months.shape

(682, 12)

In [70]:
trans_loan_final_six_months.head()

,account_id,VYBER,credit,withdrawal,total_type_count,collection_from_another_bank,credit_card_withdrawal,credit_in_cash,other,remittance_to_another_bank,withdrawal_in_cash,status
0,2,3.0,26.0,12.0,41.0,21922.333333,NaN,NaN,143.616667,7266.000000,4729.775000,0.0
1,19,0.0,20.0,22.0,42.0,16400.500000,NaN,2966.666667,77.040000,NaN,6068.650000,1.0
2,25,1.0,37.0,16.0,54.0,NaN,NaN,25006.500000,264.283333,2597.812500,11062.072727,0.0
3,37,1.0,39.0,12.0,52.0,NaN,NaN,25244.666667,117.650000,1666.000000,6104.181818,1.0
4,38,1.0,30.0,16.0,47.0,14994.500000,NaN,2025.000000,156.300000,2282.384615,4367.555556,0.0


In [71]:
trans_loan_final_six_months.isnull().sum()

account_id                        0
VYBER                             0
credit                            0
withdrawal                        0
total_type_count                  0
collection_from_another_bank    472
credit_card_withdrawal          660
credit_in_cash                  112
other                             3
remittance_to_another_bank      275
withdrawal_in_cash                3
status                            0
dtype: int64

In [72]:
trans_loan_final_six_months.dtypes

account_id                        int64
VYBER                           float64
credit                          float64
withdrawal                      float64
total_type_count                float64
collection_from_another_bank    float64
credit_card_withdrawal          float64
credit_in_cash                  float64
other                           float64
remittance_to_another_bank      float64
withdrawal_in_cash              float64
status                          float64
dtype: object

In [73]:
trans_loan_final_six_months.head()

,account_id,VYBER,credit,withdrawal,total_type_count,collection_from_another_bank,credit_card_withdrawal,credit_in_cash,other,remittance_to_another_bank,withdrawal_in_cash,status
0,2,3.0,26.0,12.0,41.0,21922.333333,NaN,NaN,143.616667,7266.000000,4729.775000,0.0
1,19,0.0,20.0,22.0,42.0,16400.500000,NaN,2966.666667,77.040000,NaN,6068.650000,1.0
2,25,1.0,37.0,16.0,54.0,NaN,NaN,25006.500000,264.283333,2597.812500,11062.072727,0.0
3,37,1.0,39.0,12.0,52.0,NaN,NaN,25244.666667,117.650000,1666.000000,6104.181818,1.0
4,38,1.0,30.0,16.0,47.0,14994.500000,NaN,2025.000000,156.300000,2282.384615,4367.555556,0.0


In [74]:
trans_loan_final_six_months['status'] = trans_loan_final_six_months.status.apply(lambda x : int(x))

In [75]:
trans_loan_final_six_months.dtypes

account_id                        int64
VYBER                           float64
credit                          float64
withdrawal                      float64
total_type_count                float64
collection_from_another_bank    float64
credit_card_withdrawal          float64
credit_in_cash                  float64
other                           float64
remittance_to_another_bank      float64
withdrawal_in_cash              float64
status                            int64
dtype: object

In [76]:
trans_loan_final_six_months.head()

,account_id,VYBER,credit,withdrawal,total_type_count,collection_from_another_bank,credit_card_withdrawal,credit_in_cash,other,remittance_to_another_bank,withdrawal_in_cash,status
0,2,3.0,26.0,12.0,41.0,21922.333333,NaN,NaN,143.616667,7266.000000,4729.775000,0
1,19,0.0,20.0,22.0,42.0,16400.500000,NaN,2966.666667,77.040000,NaN,6068.650000,1
2,25,1.0,37.0,16.0,54.0,NaN,NaN,25006.500000,264.283333,2597.812500,11062.072727,0
3,37,1.0,39.0,12.0,52.0,NaN,NaN,25244.666667,117.650000,1666.000000,6104.181818,1
4,38,1.0,30.0,16.0,47.0,14994.500000,NaN,2025.000000,156.300000,2282.384615,4367.555556,0


In [77]:
trans_loan_final_six_months[['VYBER', 'credit', 'withdrawal', 'total_type_count']] = trans_loan_final_six_months[['VYBER', 'credit', 'withdrawal', 'total_type_count']].astype(int)

In [94]:
# Load the Relation Client
client = pd.read_csv('client.asc', sep = ';')

In [95]:
client.head()

,client_id,birth_number,district_id
0,1,706213,18
1,2,450204,1
2,3,406009,1
3,4,561201,5
4,5,605703,5


In [96]:
# Get age and gender
client['birth_number']=client['birth_number'].apply(lambda x: (x-5000) if x%10000 >= 5000  else x)
client['gender'] = client['birth_number'].apply(lambda x: 'F' if x%10000 >= 5000  else 'M')

In [97]:
client.head()

,client_id,birth_number,district_id,gender
0,1,701213,18,M
1,2,450204,1,M
2,3,401009,1,M
3,4,561201,5,M
4,5,600703,5,M


In [98]:
# Prefix 19 to date of birth_number
client.birth_number = client.birth_number.apply(lambda x : '19' + str(x))

In [100]:
client['birth_number'] = client['birth_number'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d')) #Changing date format

In [102]:
# Derive Age from the birth number
client['age'] = 1999 - client.birth_number.dt.year

In [106]:
client.head()

,client_id,birth_number,district_id,gender,age
0,1,1970-12-13,18,M,29
1,2,1945-02-04,1,M,54
2,3,1940-10-09,1,M,59
3,4,1956-12-01,5,M,43
4,5,1960-07-03,5,M,39


In [104]:
# Loading Disposition Relation
disp = pd.read_csv('disp.asc', sep = ';')

In [105]:
disp.head()

,disp_id,client_id,account_id,type
0,1,1,1,OWNER
1,2,2,2,OWNER
2,3,3,2,DISPONENT
3,4,4,3,OWNER
4,5,5,3,DISPONENT


In [80]:
client.shape

(5369, 3)

In [58]:
disp.shape

(5369, 4)

# Merging Client  data and Dosposition data, table name = client_disp

In [121]:
client_disp = pd.merge(client, disp, how='left', on='client_id')

In [122]:
client_disp.head()

,client_id,birth_number,district_id,gender,age,disp_id,account_id,type
0,1,1970-12-13,18,M,29,1,1,OWNER
1,2,1945-02-04,1,M,54,2,2,OWNER
2,3,1940-10-09,1,M,59,3,2,DISPONENT
3,4,1956-12-01,5,M,43,4,3,OWNER
4,5,1960-07-03,5,M,39,5,3,DISPONENT


In [108]:
# Load the demographic
demo = pd.read_csv('district.asc', sep = ';')

In [109]:
demo.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,8507,1.67,1.85,132,2159,2674
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.95,2.21,111,2824,2813
3,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.64,5.05,109,5244,5892
4,5,Kolin,central Bohemia,95616,65,30,4,1,6,51.4,9307,3.85,4.43,118,2616,3040


In [111]:
demo.A3.value_counts()

south Moravia      14
central Bohemia    12
north Moravia      11
east Bohemia       11
west Bohemia       10
north Bohemia      10
south Bohemia       8
Prague              1
Name: A3, dtype: int64

In [117]:
demo_trim = demo[['A1', 'A3', 'A11']]

In [118]:
demo_trim.columns

Index([u'A1', u'A3', u'A11'], dtype='object')

In [119]:
demo_columns = ['district_id', 'region', 'avg_salary' ]
demo_trim.columns = demo_columns

In [120]:
demo_trim.head()

,district_id,region,avg_salary
0,1,Prague,12541
1,2,central Bohemia,8507
2,3,central Bohemia,8980
3,4,central Bohemia,9753
4,5,central Bohemia,9307


# Merging "client_disp" and "demo_trim" data, table name "client_disp_demo"

In [123]:
client_disp.head()

,client_id,birth_number,district_id,gender,age,disp_id,account_id,type
0,1,1970-12-13,18,M,29,1,1,OWNER
1,2,1945-02-04,1,M,54,2,2,OWNER
2,3,1940-10-09,1,M,59,3,2,DISPONENT
3,4,1956-12-01,5,M,43,4,3,OWNER
4,5,1960-07-03,5,M,39,5,3,DISPONENT


In [124]:
demo_trim.head()

,district_id,region,avg_salary
0,1,Prague,12541
1,2,central Bohemia,8507
2,3,central Bohemia,8980
3,4,central Bohemia,9753
4,5,central Bohemia,9307


In [125]:
client_disp_demo = pd.merge(client_disp, demo_trim, how = 'left', on = 'district_id')

In [126]:
client_disp_demo.head()

,client_id,birth_number,district_id,gender,age,disp_id,account_id,type,region,avg_salary
0,1,1970-12-13,18,M,29,1,1,OWNER,south Bohemia,8968
1,2,1945-02-04,1,M,54,2,2,OWNER,Prague,12541
2,3,1940-10-09,1,M,59,3,2,DISPONENT,Prague,12541
3,4,1956-12-01,5,M,43,4,3,OWNER,central Bohemia,9307
4,5,1960-07-03,5,M,39,5,3,DISPONENT,central Bohemia,9307


# Loading Credit_card data

In [127]:
credit = pd.read_csv('card.asc', sep = ';')

In [128]:
credit.head()

,card_id,disp_id,type,issued
0,1005,9285,classic,931107 00:00:00
1,104,588,classic,940119 00:00:00
2,747,4915,classic,940205 00:00:00
3,70,439,classic,940208 00:00:00
4,577,3687,classic,940215 00:00:00


In [129]:
credit.type.value_counts()

classic    659
junior     145
gold        88
Name: type, dtype: int64

In [130]:
credit.dtypes

card_id     int64
disp_id     int64
type       object
issued     object
dtype: object

In [131]:
client_disp_demo.dtypes

client_id                int64
birth_number    datetime64[ns]
district_id              int64
gender                  object
age                      int64
disp_id                  int64
account_id               int64
type                    object
region                  object
avg_salary               int64
dtype: object

# Merg "client_disp_demo" and "card" on "disp_id" for "card type issued to custome" final table "client_disp_demo_credit"

In [135]:
client_disp_demo_credit = pd.merge(client_disp_demo, credit, how = 'left', on = 'disp_id',suffixes=('_disposition', '_card'))

In [136]:
client_disp_demo_credit.isnull().sum()

client_id              0
birth_number           0
district_id            0
gender                 0
age                    0
disp_id                0
account_id             0
type_disposition       0
region                 0
avg_salary             0
card_id             4477
type_card           4477
issued              4477
dtype: int64

In [137]:
client_disp_demo_credit.shape

(5369, 13)

In [138]:
client_disp_demo_credit.columns

Index([u'client_id', u'birth_number', u'district_id', u'gender', u'age',
       u'disp_id', u'account_id', u'type_disposition', u'region',
       u'avg_salary', u'card_id', u'type_card', u'issued'],
      dtype='object')

In [139]:
trans_loan_final_six_months.columns

Index([u'account_id', u'VYBER', u'credit', u'withdrawal', u'total_type_count',
       u'collection_from_another_bank', u'credit_card_withdrawal',
       u'credit_in_cash', u'other', u'remittance_to_another_bank',
       u'withdrawal_in_cash', u'status'],
      dtype='object')

In [141]:
client_disp_demo_credit.shape

(5369, 13)

In [142]:
trans_loan_final_six_months.shape

(682, 12)

In [146]:
test = pd.merge(trans_loan_final_six_months,client_disp_demo_credit, how='left', on='account_id' )

In [155]:
test[['account_id','client_id', 'type_disposition']].head(100)

,account_id,client_id,type_disposition
0,2,2,OWNER
1,2,3,DISPONENT
2,19,25,OWNER
3,25,31,OWNER
4,37,45,OWNER
5,38,46,OWNER
6,67,78,OWNER
7,97,116,OWNER
8,97,117,DISPONENT
9,103,124,OWNER


In [201]:
# Display max columns
pd.options.display.max_columns = len(test.columns)

In [159]:
# Show the duplcate account row
test[['account_id', 'client_id', 'type_disposition']][test.duplicated(subset='account_id')].shape

(145, 3)

In [161]:
# Change the type of "account_id" and "client_id" to string
test.account_id = test.account_id.apply(lambda x : str(x))
test.client_id = test.client_id.apply(lambda x : str(x))

In [171]:
test.drop_duplicates(subset='account_id', keep='first', inplace=True)

In [167]:
test.columns

Index([u'account_id', u'VYBER', u'credit', u'withdrawal', u'total_type_count',
       u'collection_from_another_bank', u'credit_card_withdrawal',
       u'credit_in_cash', u'other', u'remittance_to_another_bank',
       u'withdrawal_in_cash', u'status', u'client_id', u'birth_number',
       u'district_id', u'gender', u'age', u'disp_id', u'type_disposition',
       u'region', u'avg_salary', u'card_id', u'type_card', u'issued'],
      dtype='object')

In [185]:
final_data = test[['VYBER', 'credit', 'withdrawal', 'total_type_count', 'collection_from_another_bank', 'credit_card_withdrawal'
                  ,'credit_in_cash', 'other', 'remittance_to_another_bank','withdrawal_in_cash','status','gender','age','region'
                  , 'avg_salary', 'type_card', ]]

In [186]:
final_data.head()

,VYBER,credit,withdrawal,total_type_count,collection_from_another_bank,credit_card_withdrawal,credit_in_cash,other,remittance_to_another_bank,withdrawal_in_cash,status,gender,age,region,avg_salary,type_card
0,3,26,12,41,21922.333333,NaN,NaN,143.616667,7266.000000,4729.775000,0,M,54,Prague,12541,NaN
2,0,20,22,42,16400.500000,NaN,2966.666667,77.040000,NaN,6068.650000,1,M,60,south Bohemia,9104,NaN
3,1,37,16,54,NaN,NaN,25006.500000,264.283333,2597.812500,11062.072727,0,M,37,north Moravia,9893,NaN
4,1,39,12,52,NaN,NaN,25244.666667,117.650000,1666.000000,6104.181818,1,M,47,south Bohemia,8547,NaN
5,1,30,16,47,14994.500000,NaN,2025.000000,156.300000,2282.384615,4367.555556,0,M,59,south Bohemia,8402,NaN


In [189]:
final_data.collection_from_another_bank.fillna(value = 0, inplace = True)

/usr/local/lib/python2.7/dist-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [198]:
final_data.isnull().sum()

VYBER                             0
credit                            0
withdrawal                        0
total_type_count                  0
collection_from_another_bank      0
credit_card_withdrawal            0
credit_in_cash                    0
other                             0
remittance_to_another_bank        0
withdrawal_in_cash                0
status                            0
gender                            0
age                               0
region                            0
avg_salary                        0
type_card                       512
dtype: int64

In [191]:
# Fill all the missing numerical value
final_data.credit_card_withdrawal.fillna(value = 0, inplace = True)
final_data.remittance_to_another_bank.fillna(value = 0, inplace = True)
final_data.withdrawal_in_cash.fillna(value = 0, inplace = True)
final_data.credit_in_cash.fillna(value = 0, inplace = True)
final_data.other.fillna(value =0, inplace = True)

In [200]:
# Fill type_card with "no_card"
final_data.type_card.fillna(value = 'no_card', inplace = True)

In [204]:
final_data.to_csv('final_data.csv', index = False)